In [9]:
import tweepy #pip install tweepy if no module found - OR If you are using Anaconda try $ conda install -c conda-forge tweepy
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter
import sys

#uncomment below to start from today, if not specify a date (YYYY, M, D)
#DATE = datetime.now()
DATE = datetime(2018, 4, 4)

date = DATE

datestrTodayFile = str(date.day)+ "-" + str(date.month) + "-" +  str(date.year) #file datestring

print("Gathering tweets for " + datestrTodayFile)

#today auth tweepy format
date = datetime.now()
datestrToday = str(date.year) + "-" + str(date.month) + "-" + str(date.day)

#yesterday auth tweepy format
date = date - timedelta(1)
datestrYesterday = str(date.year) + "-" + str(date.month) + "-" + str(date.day)


## Insert your API keys and tokens below, go to https://apps.twitter.com and create an app if you need one
#Consumer Key (API Key)
consumerKey = "t681OQY2eRAQb1202nm4v6gTq"
#Consumer Secret (API Secret)
consumerSecret = "l4A0rVbskUszJX6Ln7z3jGSQ7Bnq4Rywjp9vIDw1XFB7zvkBS1"
#Access Token
accessToken = "974576552670212096-1QYUN2KGt5XdXuaYoVfddmXqNxn6OQV"
#Access Token Secret
accessTokenSecret = "RPxjTMAD6hJ5xf3j9KMA5fe5KGoMCMkpYgPr1ufBxQO9v"

auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)

MAX_TWEETS = 2000

api = tweepy.API(auth)


# predefined dictionaries as features - this is looking at illness - perhaps we can detect a disease outbreak
FEATURES=["sick", "ill", "not feeling well", "sick at home", "cough", "fever", "headache", "doctors", "getting sick", "feeling sick", "sickie", "sick day", "a cold", "the flu", "blood test", "hospital bed", "flu", "tonsilitis", "locum"]

#Search geotagged tweets which contain certain features
places = api.geo_search(query="Australia", granularity="country", since=datestrYesterday, until=datestrToday)
place_id = places[0].id

## create long query string
queries = []
for word in FEATURES:
    queries.append("place:%s" % place_id +  " \"" + word + "\"")

Gathering tweets for 4-4-2018


In [10]:
import csv   
import os
## setup CSV
## create one for today if it doesn't exist
filename = 'Tweets/' + datestrTodayFile+'.csv'

api = tweepy.API(auth)
places = api.geo_search(query="Australia", granularity="country", since=datestrYesterday, until=datestrToday)
place_id = places[0].id

## gets all the tweets for the features definated above and saves to a csv
def save_tweets_to_csv(): 
    for query in queries:
        print("Gathering tweets for " + query)
        
        if os.path.exists(filename):
            append_write = 'a' # append if already exists
        else:
            append_write = 'w' # make a new file if not
        
        try:
            tweets = api.search(q=query, tweet_mode='extended')

            with open(filename, append_write, newline='', encoding='utf-8') as c: 
                writer = csv.writer(c)
                if append_write == 'w': # new file
                    writer.writerow(["text", "id"]) #append column title
                    
                count = 0 # number of tweets found in this feature

                for tweet in tweets:

                    ## ensure string is one one line, for proper compliation
                    tweetText = tweet.full_text.replace('\n', ' ').replace('\r', '')

                    writer.writerow([tweetText + tweet.place.name if tweet.place else "", tweet.id])
                    print ("feature: " + query + ", " + tweet.full_text + " | " + tweet.place.name if tweet.place else "Undefined place \n") 
                    count = count + 1
            
            print("Found " + str(count) + " tweets containing /'" + query + "'/")
        except:
            print("Too many requests, waiting 15 minutes..") #twitter 88 error code - wait 15 minutes
            time.sleep(60 * 15) # wait 15 minutes

In [11]:
## gets all the tweets for the features definated above and returns a list
def get_tweets(): 
    
    allTweets = []
    for query in queries:
        print("Gathering tweets for " + query)
        
        try:
            tweets = api.search(q=query, tweet_mode='extended')
                    
            count = 0 # number of tweets found in this feature

            for tweet in tweets:

                ## ensure string is one one line, for proper compliation
                tweetText = tweet.full_text.replace('\n', ' ').replace('\r', '')

                # append location information also
                allTweets.append(tweetText + tweet.place.name if tweet.place else "")
                
                print ("feature: " + query + ", " + tweet.full_text + " | " + tweet.place.name if tweet.place else "Undefined place \n") 
                count = count + 1
            
            print("Found " + str(count) + " tweets containing /'" + query + "'/")
        except:
            print("Too many requests, waiting 15 minutes..") #twitter 88 error code - wait 15 minutes
            time.sleep(60 * 15) # wait 15 minutes
            
    return allTweets

tweets = get_tweets() #grab a list of tweets

Gathering tweets for place:3f14ce28dc7c4566 "sick"
feature: place:3f14ce28dc7c4566 "sick", @davidjo99414992 @Hally2289 @Fascistlefty @chriskkenny @adambspencer Owned by the French Engie Co.(72%) &amp; the Japanese Mitsui &amp; Co.(28%) Hazelwood air pollution had been estimated to kill at least 18 people pa in Gippsland, and many more sick. Estimated health burden on the community was $100m pa, a huge subsidy. https://t.co/LJcfKakVpm | Canberra
feature: place:3f14ce28dc7c4566 "sick", @2GB873 Watched the first 20 minutes, as an Australian I felt sick and hurt, and discriminated,why am I so different.
This is my Country. I do not look like a stone aged person! | Gold Coast
feature: place:3f14ce28dc7c4566 "sick", @IndianaRose16 So sorry to hear Indiana you are still sick take care 😘😘😘 | Perth (WA)
feature: place:3f14ce28dc7c4566 "sick", Been asked to go watch the new Tomb Raider. I'm sick. | Brisbane
feature: place:3f14ce28dc7c4566 "sick", @AlisonCostelloe @RACTOfficial You bet 👍 sick of 

feature: place:3f14ce28dc7c4566 "fever", Very excited to announce, with just 2 tickets left for Nocturnal Fever, the beautiful… https://t.co/b3rTDbmvkv | Melbourne
feature: place:3f14ce28dc7c4566 "fever", I would definitely recommend adulthood glandular fever if you have an otherwise good immune system and plenty of sick leave to burn. I spent about 4 days proper sick and the other week off work playing Terraria. Also lost a couple kg. Will sell my contagious saliva for a price. | The Basin
feature: place:3f14ce28dc7c4566 "fever", @troygrant is there somethng wrong with the team at Mudgee hospital when they cannot diagnose a child rash or fever and have them sent 70 ks to Dubbo hospital ?? my granddaughter had this happen today and is still out there ! So I like an answer as is this Y 70 mill been spent | Sydney
feature: place:3f14ce28dc7c4566 "fever", Q. What is the best part of a #CommonwealthGames’ boxer’s joke?
A. The punch line.

The #GoldCoast is all a-buzz with #GC2018 fever thi

feature: place:3f14ce28dc7c4566 "feeling sick", A doctor w/ Indian Boxing team will be issued a strong written reprimand for breaches of #GC2018 no needles policy.  Needle contained a vitamin B complex which was administered to an athlete feeling sick  #TenNews | Gold Coast
feature: place:3f14ce28dc7c4566 "feeling sick", @Mazer_9 This does not need be shared, it needs to be BANNED.. I wish it hadn’t appeared on my TL. Feeling sick. | Canberra
feature: place:3f14ce28dc7c4566 "feeling sick", So, let’s say you were feeling sick. What’s the absolute maximum number of painkillers one can take without dying? Because I have at least a 50 hour week coming up. If I’m lucky. #asktwitter #🤒 | Melbourne
feature: place:3f14ce28dc7c4566 "feeling sick", Happy Easter Everyone! I know I will be feeling sick tonight from eating too much chocolate 🍫 😅

@ScarletBlue9 https://t.co/AxFhwrsnAv | Sydney
feature: place:3f14ce28dc7c4566 "feeling sick", Typical...start feeling sick just days away from mini getaw

feature: place:3f14ce28dc7c4566 "blood test", In 6 minutes, I have to start fasting because of my blood test tomorrow. 😪 | Sydney
feature: place:3f14ce28dc7c4566 "blood test", Canardly wait for the blood test. https://t.co/DCU3JnajCq | Sydney
feature: place:3f14ce28dc7c4566 "blood test", I just had a blood test and the lady asked if I was on holidays and I said yes and she then asked me if I was at uni and I was like uhhh I’m 15 she apologized so much it was so funny and then she goes “oh geez I thought because you were here on your own you probably drove here” | Melbourne
feature: place:3f14ce28dc7c4566 "blood test", Well, I’m getting a blood test on Thursday for many things and I’m scared what the results could b. Will I get an urgent call to come back like my sister or will I just come in a couple days later because the levels aren’t that worrying... | Sydney
feature: place:3f14ce28dc7c4566 "blood test", Oh they developed a new blood test to detect alzeimers 30 years in advance. 

G

In [13]:
#Do some basic K Means clustering on the tweets produced

from sklearn.feature_extraction.text import TfidfVectorizer #pip install sklearn
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

NUM_CLUSTERS = 3
MAX_ITERATIONS = 100
NUM_INIT = 3
NUM_TERMS_PER_CLUSTER = 10

#vectorize the tweets (remove stopwords convert the strings to numeric features)
vectorizer = TfidfVectorizer(stop_words='english', smooth_idf=True)
vectorizedTweets = vectorizer.fit_transform(tweets)
print(vectorizedTweets)

#cluster tweets
cluteringModel = KMeans(n_clusters=NUM_CLUSTERS, init='k-means++', max_iter=MAX_ITERATIONS, n_init=NUM_INIT)
cluteringModel.fit(vectorizedTweets)

#print top terms per cluster clusters
print("Top terms per cluster:")
order_centroids = cluteringModel.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(NUM_CLUSTERS):
    print ("Cluster %d:" % i)
    for ind in order_centroids[i, :NUM_TERMS_PER_CLUSTER]:
        print (' %s' % terms[ind])

  (0, 271)	0.1694194300027796
  (0, 466)	0.1694194300027796
  (0, 371)	0.1694194300027796
  (0, 204)	0.1694194300027796
  (0, 45)	0.1694194300027796
  (0, 773)	0.1694194300027796
  (0, 408)	0.1694194300027796
  (0, 342)	0.1694194300027796
  (0, 27)	0.1694194300027796
  (0, 76)	0.22993467236449605
  (0, 573)	0.1694194300027796
  (0, 699)	0.1694194300027796
  (0, 13)	0.1694194300027796
  (0, 478)	0.1694194300027796
  (0, 66)	0.15646829416741545
  (0, 814)	0.15646829416741545
  (0, 353)	0.3388388600055592
  (0, 593)	0.1694194300027796
  (0, 6)	0.1694194300027796
  (0, 791)	0.11801167972859702
  (0, 774)	0.3388388600055592
  (0, 430)	0.1694194300027796
  (0, 934)	0.06660392945441448
  (0, 484)	0.1472793170563981
  (0, 169)	0.1694194300027796
  :	:
  (145, 396)	0.3595429995073994
  (145, 1003)	0.18555182551751787
  (145, 292)	0.1662024956479048
  (145, 754)	0.18555182551751787
  (145, 1182)	0.18555182551751787
  (145, 835)	0.18555182551751787
  (145, 907)	0.18555182551751787
  (145, 216)	0.